-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Trabalho Final Optativa I - Tópicos Especiais de Informática (Ciências de Dados)
Aluno: Carlos Henrique Pinheiro Cordeiro

Este trabalho tem como objetivo exercitar os conceitos práticos que envoltam a Ciência de Dados, aplicando métodos que um cientista de dados pratica no seu dia a dia, como a análise do contexto que se procura uma resposta para uma dada pergunta, a captura dos dados, a limpeza destes mesmos dados, e por fim o uso das tecnologias necessárias para atingir seu objetivo.

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 1. Abertura e limpeza dos dados

Primeiramente é necessário abrir os arquivos contendo os dados necessários para a resolução do exercício. Os arquivos contendo os dados referentes à quantidade de chuva  (mm) e o nível da água (cm), foram renomeados para serem mais simples de abrir e identificar quais destes contém o necessário para o dado momento. Receberam nomes com prefixo que anuncia qual dado está presente, se é chuva ou nível, seguido do nome da cidade do qual foram obtidos estes dados. Por exemplo: "chuva-taio.txt" contempla os dados dos níveis de chuva da cidade de Taió.

Os arquivos foram abertos e salvos em variáveis de "dataset", para depois estes datasets passarem por processos de limpeza dos seus dados, sendo preparados para o treinamento do modelo que nos fará atingir nosso objetivo, que será abordado na próxima seção.

De longe, esta foi a parte mais difícil de se realizar e planejar. A questão era pensar como os arquivos seriam limpos de modo que seja fácil de abri-los como DataSets pelo Pandas. Com o auxílio do software VSCode e suas ferramentas (nativas) de substituição de caracteres, foi possível tratar os arquivos de txt como se fossem csv's (comma separated values - valores separados por vírgula). Sendo assim, bastou apenas abrir os arquivos por aqui pelo Pandas, e tratar as colunas de cada um. Após, o desafio foi planejar como estes dados poderiam ser unidos e "conversariam". Para tal, foi decidido juntar os DataSets através de um "merge" pela Data e Hora, em um DataSet geral. Bastando que depois apenas fosse realizado um tratamento final de valores nulos e reinicialização de índice do DataSet, para uma melhor leitura de seus dados. Sendo assim, estamos prontos para o próximo passo.

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 1.1. Implementação da Abertura e Limpeza dos dados

In [ ]:
import pandas as pd

#Função para abrir um arquivo de txt com os dados, apenas informando de qual cidade quero o arquivo
def abreArquivoNivel(cidade):
    nomeArquivo = 'nivel-'+cidade+'.txt'
    df = pd.read_csv(nomeArquivo, sep=",", encoding="UTF-8", names=[cidade+'-un1', 'data', 'hora', cidade+'Nivel', cidade+'-un2'])
    df = df.drop(columns=[cidade+'-un1', cidade+'-un2'])
    return df

#Abertura dos arquivos por cidade
nivelTaio       = abreArquivoNivel("taio")
nivelRioDoSul   = abreArquivoNivel("riodosul")
nivelItuporanga = abreArquivoNivel("ituporanga")

#Criação do DataSet geral
dadosGerais = pd.merge(nivelRioDoSul, nivelTaio      , how='left', on=['data', 'hora'])
dadosGerais = pd.merge(dadosGerais  , nivelItuporanga, how='left', on=['data', 'hora'])
dadosGerais = dadosGerais.dropna()
dadosGerais = dadosGerais.reset_index()
dadosGerais = dadosGerais.drop(columns='index')
print(dadosGerais)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 2. Formulação do objetivo

Tendo em mãos os dados, já limpos, das chuvas e níveis das cidades de Rio do Sul, Ituporanga e Taió, sendo os valores correspondendes à Data, Hora e a respectiva medida, podemos estabelecer um objetivo para alcançar, a partir destes dados utilizados no treinamento de algum modelo de Aprendizado de Máquina.

O objetivo é treinar um modelo de aprendizado de máquina que possa nos dar uma previsão do nível do rio de Rio do Sul, de acordo com o nível dos rios de Ituporanga e Taió. Sendo assim, daremos a ele o nível dos rios de Ituporanga e Taió, e ele nos dará qual será o nível do rio de Rio do Sul.

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 2.1. Montagem do DataSet para treinamento

In [69]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#Montagem dos dados para treinamento
x = dadosGerais[['taioNivel', 'ituporangaNivel']]
y = dadosGerais['riodosulNivel']

#Separando o conjunto de dados em um conjunto para treinamento e outro para testes
xTreinamento, xTeste, yTreinamento, yTeste = train_test_split(x, y, train_size = 0.8, test_size = 0.2)

#Treinamento do modelo com os dados
modelo = LinearRegression()
modelo.fit(xTreinamento, yTreinamento)

#Previsão
previsaoCheia = modelo.predict(xTeste)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 3. Métricas de erro do modelo

Passada uma previsão de cheia dando os níveis de Taió e Ituporanga, temos as métricas de erro relacionadas ao modelo, observando por aqui: R-Quadrado, R-Quadrado Ajustado, Erro Quadrático Médio ou MSE (por mais que tenha suas desvantagens, como veremos na diferença dos resultados deste para os do RMSE), Raíz do Erro Quadrático Médio (RMSE), Erro Percentual Absoluto Médio (MAPE), e por último, uma tabela de correlação por cores.

Assim, finalizando o trabalho, empregando o uso das tecnologias apropriadas junto das técnicas e práticas exercidas, sendo esta mistura ensinada em sala de aula, assim exercendo um estudo e exercício pragmático da Ciência de Dados.

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np

#R²
R2 = r2_score(yTeste, previsaoCheia)
print('R²: ',R2)

#R² Ajustado
R2Ajustado = (1 - ((1 - r2_score(yTeste, previsaoCheia)) * (len(yTeste) - 1)) / 
            (len(yTeste) - xTeste.shape[1] - 1))
print('R² Ajustado: ', R2Ajustado)

#Erro Quadrático Médio (MSE)
MSE = mean_squared_error(yTeste, previsaoCheia)
print("Erro Quadrático Médio: %.2f" %MSE)

#Raíz do Erro Quadrático Médio (RMSE)
RMSE = mean_squared_error(yTeste, previsaoCheia, squared=False) 
print("Raíz do Erro Quadrático Médio: %.2f" %RMSE)

#Erro Percentual Absoluto Médio (MAPE)
MAPE = np.mean(np.abs((yTeste - previsaoCheia) / yTeste)) * 100
print("Erro Percentual Absoluto Médio: %.2f" %MAPE+"%")

#Tabela de Correlação por Cores
tabela = dadosGerais.corr()
tabela.style.background_gradient(cmap='coolwarm').set_precision(2)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------